<a href="https://colab.research.google.com/github/ShounakDas101/AIML_Hari/blob/main/ML4SCI_ViTBinaryClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import h5py
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision import transforms
import torch.optim as optim

import matplotlib.pyplot as plt

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#DATA_DIR='./data'

DATA_DIR= '/content/drive/MyDrive/'

IMAGE_SIZE = 32 # W=32 H=32
NUM_CHANNELS = 2 # R G B then it should be 3
PATCH_SIZE = 8 # W_patch=4 H_patch=4
NUM_PATCHES = (IMAGE_SIZE//PATCH_SIZE)*(IMAGE_SIZE//PATCH_SIZE) # = 64
NUM_HEADS = 8  # enbedding dim should be divisible by no of heads
EMBBEDING_DIM = PATCH_SIZE*PATCH_SIZE*NUM_CHANNELS  #NUM_PATCHNELS
MLP_SIZE = 512
NUM_LAYERS = 8

NUM_CLASSES = 2
NUM_WORKERS = 1
BATCH_SIZE = 512
EPOCHS = 40
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-1

In [ ]:
''' for Laptop only
# clearing cuda cache memory
import gc
torch.cuda.empty_cache()
gc.collect()
'''

In [7]:
import os

# Specify the path to the file in your Google Drive
file_path = '/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
# CHECKPOINT_PATH = '/content/drive/MyDrive/CHECKPOINTS'

# Check if the file exists
if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
else:
    print(f"The file '{file_path}' does not exist.")

The file '/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5' exists.


In [8]:
import h5py

def find_keys(hdf5_obj, path='/'):
    """Recursively find keys in an HDF5 file."""
    keys = []
    for key in hdf5_obj[path].keys():
        full_path = f"{path}/{key}"
        keys.append(full_path)
        if isinstance(hdf5_obj[full_path], h5py.Group):
            keys.extend(find_keys(hdf5_obj, full_path))
    return keys

# Open the HDF5 file
file_path = '/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'  # Replace with your actual file path
with h5py.File(file_path, 'r') as hdf5_file:
    # Find keys starting from the root
    all_keys = find_keys(hdf5_file)

# Print the found keys
for key in all_keys:
    print(key)

//X
//y


In [9]:
# import dataset

# importing electron dataset and seperating images and labels
electron_dataset = h5py.File('/content/drive/MyDrive/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5',"r")
electron_imgs=np.array(electron_dataset["X"])
electron_labels=np.array(electron_dataset["y"],dtype=np.int64)

# importing photon dataset and seperating images and labels
photon_dataset = h5py.File('/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5',"r")
photon_imgs=np.array(photon_dataset["X"])
photon_labels=np.array(photon_dataset["y"],dtype=np.int64)

In [10]:
print(electron_imgs.shape)
print(electron_imgs.dtype)
print(electron_labels)
print(photon_imgs.shape)
print(photon_labels)

(249000, 32, 32, 2)
float32
[1 1 1 ... 1 1 1]
(249000, 32, 32, 2)
[0 0 0 ... 0 0 0]


In [11]:
# concatenate electron and photon images/labels
img_arrs = torch.Tensor(np.vstack((photon_imgs,electron_imgs)))
labels = torch.Tensor(np.hstack((photon_labels,electron_labels))).to(torch.int64)
print(img_arrs.shape)
img_arrs = img_arrs.permute(0,3,1,2)
print(img_arrs.shape)

torch.Size([498000, 32, 32, 2])
torch.Size([498000, 2, 32, 32])


In [12]:
del electron_imgs,photon_imgs,electron_labels,photon_labels

In [13]:
from torch.utils.data import Dataset, DataLoader, random_split

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'image': self.data[idx], 'label': self.labels[idx]}
        return sample

In [14]:
# Assuming img_arrs and labels are already defined
custom_dataset = CustomDataset(img_arrs, labels)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(custom_dataset))
valid_size = int(0.2 * len(custom_dataset))
test_size = len(custom_dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(custom_dataset, [train_size, valid_size, test_size])
print(len(valid_dataset)+len(test_dataset)+len(train_dataset))

498000


In [15]:
# Create DataLoaders for training, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                                           num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                                           num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False,
                                          num_workers=NUM_WORKERS, pin_memory=True)

In [16]:
# next(iter(...)) catches the first batch of the data loader
# If shuffle is True, this will return a different batch every time we run this cell
# For iterating over the whole dataset, we can simple use "for batch in data_loader: ..."
data_inputs = next(iter(train_loader))

# The shape of the outputs are [batch_size, d_1,...,d_N] where d_1,...,d_N are the
# dimensions of the data point returned from the dataset class
print("Data inputs", data_inputs['image'].shape, "\n")
# print("Full",data_inputs)

Data inputs torch.Size([512, 2, 32, 32]) 



In [17]:
# 1. Create a class which subclasses nn.Module
class PatchEmbedding(nn.Module):
    """Turns a 2D input image into a 1D sequence learnable embedding vector.

    Args:
        in_channels (int): Number of color channels for the input images. Defaults to 3.
        patch_size (int): Size of patches to convert input image into. Defaults to 16.
        embedding_dim (int): Size of embedding to turn image into. Defaults to 768.
    """
    # 2. Initialize the class with appropriate variables
    def __init__(self,
                 in_channels:int=NUM_CHANNELS, # for images n_channels:int=3
                 patch_size:int=PATCH_SIZE, #
                 embedding_dim:int=EMBBEDING_DIM):
        super().__init__()

        # 3. Create a layer to turn an image into patches

        self.patcher = nn.Conv2d(in_channels=1, # each channel we will process separately
                                 out_channels=embedding_dim//2,
                                 kernel_size=patch_size,
                                 stride=patch_size,
                                 padding=0)



        '''
        # Cut Images into patches
        patches = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size).reshape((b, int((w / self.patch_size) * (h / self.patch_size)), self.patch_size ** 2))
        patches_rolled = x_clone.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size).reshape((b, int((w / self.patch_size) * (h / self.patch_size)), self.patch_size ** 2))
        x = torch.cat((patches, patches_rolled), 1)
        x = torch.relu(self.linear1(x))
        '''
        # 4. Create a layer to flatten the patch feature maps into a single dimension
        self.flatten = nn.Flatten(start_dim=2, # only flatten the feature map dimensions into a single vector
                                  end_dim=3)

    # 5. Define the forward method
    def forward(self, x):
        '''
        # Example tensor
        x = torch.randn(2, 3, 4, 4)  # Assuming a tensor with 3 channels

        # Separate channels into individual tensors
        channel1 = x[:, 0:1, :, :]  # Extract first channel
        channel2 = x[:, 1:2, :, :]  # Extract second channel
        channel3 = x[:, 2:3, :, :]  # Extract third channel (if any)
        '''
        # Create assertion to check that inputs are the correct shape
        image_resolution = x.shape[-1]
        assert image_resolution % PATCH_SIZE == 0, f"Input image size must be divisble by patch size, image shape: {image_resolution}, patch size: {patch_size}"

        channel1 = x[:, 0:1, :, :]  # Extract first channel
        channel2 = x[:, 1:2, :, :]  # Extract first channel
        #print(channel1.shape)
        # Perform the forward pass for channel 1 & 2
        channel1_patched = self.patcher(channel1)
        channel2_patched = self.patcher(channel2)
        #print(channel1_patched.shape)
        channel1_flattened = self.flatten(channel1_patched)
        channel2_flattened = self.flatten(channel2_patched)
        #print(channel1_flattened.shape)
        channel1_flattened=channel1_flattened.permute(0,2,1)
        channel2_flattened=channel2_flattened.permute(0,2,1)
        #print("after permute",channel1_flattened.shape)
        result = torch.cat((channel1_flattened, channel2_flattened), dim=2)

        ''' Old Codes................
        # Perform the forward pass
        #x = x.unfold(2,PATCH_SIZE,PATCH_SIZE).unfold(3,PATCH_SIZE,PATCH_SIZE)
        print(x.shape)
        x_patched = self.patcher(x)  # ....................>>>>>
        print(x_patched.shape)
        x_flattened = self.flatten(x_patched)
        print(x_flattened.shape)
        # 6. Make sure the output shape has the right order
        return x_flattened.permute(0, 2, 1) # adjust so the embedding is on the final dimension [batch_size, P^2•C
        '''

        #print(result.shape)
        return result

In [18]:
# Let's test it on single image
# patch_size =4

# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.


    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)



set_seeds()

# Create an instance of patch embedding layer
patchify = PatchEmbedding(in_channels=NUM_CHANNELS, patch_size=PATCH_SIZE, embedding_dim=EMBBEDING_DIM)
#patchify = PatchEmbedding(patch_size=PATCH_SIZE)
print(data_inputs['image'].shape)
# Pass a single image through
patch_embedded_image = patchify(data_inputs['image']) # add an extra batch dimension on the 0th index, otherwise will error

# View the patch embedding and patch embedding shape
# print(patch_embedded_image)
print(f"Patch embedding shape: {patch_embedded_image.shape} -> [batch_size, number_of_patches, embedding_dimension]")


torch.Size([512, 2, 32, 32])
Patch embedding shape: torch.Size([512, 16, 128]) -> [batch_size, number_of_patches, embedding_dimension]


In [19]:
# Now add the the learnable class embedding and position embeddings
# From start to positional encoding: All in 1 cell

set_seeds()

# 1. Set patch size
#patch_size = 8 # already set in hyperparameter block

# 2. Print shape of original image tensor and get the image dimensions
print(f"Image tensor shape: {data_inputs['image'].shape}")
height, width = data_inputs['image'].shape[2], data_inputs['image'].shape[3]

# 3. Get image tensor and add batch dimension
# x = image.unsqueeze(0)
print(f"Input image with batch dimension shape: {data_inputs['image'].shape}")

# 4. Create patch embedding layer
patch_embedding_layer = PatchEmbedding(in_channels=NUM_CHANNELS,
                                       patch_size=PATCH_SIZE,
                                       embedding_dim=EMBBEDING_DIM)

# 5. Pass image through patch embedding layer
patch_embedding = patch_embedding_layer(data_inputs['image'])
print(f"Patching embedding shape: {patch_embedding.shape}")



# 6. Create class token embedding
batch_size = patch_embedding.shape[0]
embedding_dimension = patch_embedding.shape[-1]
class_token = nn.Parameter(torch.rand(batch_size, 1, embedding_dimension),
                           requires_grad=True) # make sure it's learnable
print(f"Class token embedding shape: {class_token.shape}")

# 7. Prepend class token embedding to patch embedding
patch_embedding_class_token = torch.cat((class_token, patch_embedding), dim=1)
print(f"Patch embedding with class token shape: {patch_embedding_class_token.shape}")

# 8. Create position embedding
number_of_patches = NUM_PATCHES #int((height * width) / PATCH_SIZE**2)
position_embedding = nn.Parameter(torch.ones(1, number_of_patches+1, embedding_dimension),
                                  requires_grad=True) # make sure it's learnable


# 9. Add position embedding to patch embedding with class token
patch_and_position_embedding = patch_embedding_class_token + position_embedding
print(f"Patch and position embedding shape: {patch_and_position_embedding.shape}")
#patch_and_position_embedding

print(patch_embedding_class_token)  #1 is added in the beginning of each

Image tensor shape: torch.Size([512, 2, 32, 32])
Input image with batch dimension shape: torch.Size([512, 2, 32, 32])
Patching embedding shape: torch.Size([512, 16, 128])
Class token embedding shape: torch.Size([512, 1, 128])
Patch embedding with class token shape: torch.Size([512, 17, 128])
Patch and position embedding shape: torch.Size([512, 17, 128])
tensor([[[ 0.9549,  0.4435,  0.6924,  ...,  0.4531,  0.4736,  0.9448],
         [-0.0886,  0.1006,  0.1054,  ..., -0.0783,  0.0338,  0.0858],
         [-0.0886,  0.1006,  0.1054,  ..., -0.0783,  0.0338,  0.0858],
         ...,
         [-0.0886,  0.1006,  0.1054,  ..., -0.0783,  0.0338,  0.0858],
         [-0.0886,  0.1006,  0.1054,  ..., -0.0783,  0.0338,  0.0858],
         [-0.0886,  0.1006,  0.1054,  ..., -0.0783,  0.0338,  0.0858]],

        [[ 0.6190,  0.7318,  0.3390,  ...,  0.6282,  0.7355,  0.1682],
         [-0.0886,  0.1006,  0.1054,  ..., -0.0783,  0.0338,  0.0858],
         [-0.0886,  0.1006,  0.1054,  ..., -0.0783,  0.0338,

In [20]:
# 1. Create a class that inherits from nn.Module
class MultiheadSelfAttentionBlock(nn.Module):
    """Creates a multi-head self-attention block ("MSA block" for short).
    """
    # 2. Initialize the class with hyperparameters from Table 1
    def __init__(self,
                 embedding_dim:int=EMBBEDING_DIM, # Hidden size D from Table 1 for ViT-Base
                 num_heads:int=NUM_HEADS, # Heads from Table 1 for ViT-Base
                 attn_dropout:float=0): # doesn't look like the paper uses any dropout in MSABlocks
        super().__init__()

        # 3. Create the Norm layer (LN)
        self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

        # 4. Create the Multi-Head Attention (MSA) layer
        self.multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim,
                                                    num_heads=num_heads,
                                                    dropout=attn_dropout,
                                                    batch_first=True) # does our batch dimension come first?

    # 5. Create a forward() method to pass the data throguh the layers
    def forward(self, x):
        x = self.layer_norm(x)
        attn_output, _ = self.multihead_attn(query=x, # query embeddings
                                             key=x, # key embeddings
                                             value=x, # value embeddings
                                             need_weights=False) # do we need the weights or just the layer outputs?
        return attn_output

In [21]:
# 1. Create a class that inherits from nn.Module
class MLPBlock(nn.Module):
    """Creates a layer normalized multilayer perceptron block ("MLP block" for short)."""
    # 2. Initialize the class with hyperparameters from Table 1 and Table 3
    def __init__(self,
                 embedding_dim:int=EMBBEDING_DIM, # Hidden Size D from Table 1 for ViT-Base
                 mlp_size:int=MLP_SIZE, # MLP size from Table 1 for ViT-Base
                 dropout:float=0.1): # Dropout from Table 3 for ViT-Base
        super().__init__()

        # 3. Create the Norm layer (LN)
        self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

        # 4. Create the Multilayer perceptron (MLP) layer(s)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=embedding_dim,
                      out_features=mlp_size),
            nn.GELU(), # "The MLP contains two layers with a GELU non-linearity (section 3.1)."
            nn.Dropout(p=dropout),
            nn.Linear(in_features=mlp_size, # needs to take same in_features as out_features of layer above
                      out_features=embedding_dim), # take back to embedding_dim
            nn.Dropout(p=dropout) # "Dropout, when used, is applied after every dense layer.."
        )

    # 5. Create a forward() method to pass the data throguh the layers
    def forward(self, x):
        x = self.layer_norm(x)
        x = self.mlp(x)
        return x


In [22]:
# 1. Create a class that inherits from nn.Module
class TransformerEncoderBlock(nn.Module):
    """Creates a Transformer Encoder block."""
    # 2. Initialize the class with hyperparameters from Table 1 and Table 3
    def __init__(self,
                 embedding_dim:int=EMBBEDING_DIM, # Hidden size D from Table 1 for ViT-Base
                 num_heads:int=NUM_HEADS, # Heads from Table 1 for ViT-Base
                 mlp_size:int=MLP_SIZE, # MLP size from Table 1 for ViT-Base
                 mlp_dropout:float=0.1, # Amount of dropout for dense layers from Table 3 for ViT-Base
                 attn_dropout:float=0): # Amount of dropout for attention layers
        super().__init__()

        # 3. Create MSA block (equation 2)
        self.msa_block = MultiheadSelfAttentionBlock(embedding_dim=embedding_dim,
                                                     num_heads=num_heads,
                                                     attn_dropout=attn_dropout)

        # 4. Create MLP block (equation 3)
        self.mlp_block =  MLPBlock(embedding_dim=embedding_dim,
                                   mlp_size=mlp_size,
                                   dropout=mlp_dropout)

    # 5. Create a forward() method
    def forward(self, x):

        # 6. Create residual connection for MSA block (add the input to the output)
        x =  self.msa_block(x) + x

        # 7. Create residual connection for MLP block (add the input to the output)
        x = self.mlp_block(x) + x

        return x

In [ ]:
transformer_encoder_block = TransformerEncoderBlock()
from torchinfo import summary
# # Print an input and output summary of our Transformer Encoder (uncomment for full output)
summary(model=transformer_encoder_block,
        input_size=(BATCH_SIZE, PATCH_SIZE*PATCH_SIZE, EMBBEDING_DIM), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
       row_settings=["var_names"])

In [23]:
# 1. Create a ViT class that inherits from nn.Module
class ViT(nn.Module):
    """Creates a Vision Transformer architecture with ViT-Base hyperparameters by default."""
    # 2. Initialize the class with hyperparameters from Table 1 and Table 3
    def __init__(self,
                 img_size:int=IMAGE_SIZE, # Training resolution from Table 3 in ViT paper
                 in_channels:int=NUM_CHANNELS, # Number of channels in input image
                 patch_size:int=PATCH_SIZE, # Patch size
                 num_transformer_layers:int=NUM_LAYERS, # Layers from Table 1 for ViT-Base
                 embedding_dim:int=EMBBEDING_DIM, # Hidden size D from Table 1 for ViT-Base
                 mlp_size:int=MLP_SIZE, # MLP size from Table 1 for ViT-Base
                 num_heads:int=NUM_HEADS, # Heads from Table 1 for ViT-Base
                 attn_dropout:float=0, # Dropout for attention projection
                 mlp_dropout:float=0.1, # Dropout for dense/MLP layers
                 embedding_dropout:float=0.1, # Dropout for patch and position embeddings
                 num_classes:int=NUM_CLASSES): # Default for ImageNet but can customize this
        super().__init__() # don't forget the super().__init__()!

        # 3. Make the image size is divisble by the patch size
        assert img_size % patch_size == 0, f"Image size must be divisible by patch size, image size: {img_size}, patch size: {patch_size}."

        # 4. Calculate number of patches (height * width/patch^2)
        self.num_patches = NUM_PATCHES #(img_size * img_size) // patch_size**2

        # 5. Create learnable class embedding (needs to go at front of sequence of patch embeddings)
        self.class_embedding = nn.Parameter(data=torch.randn(1, 1, embedding_dim),
                                            requires_grad=True)

        # 6. Create learnable position embedding
        self.position_embedding = nn.Parameter(data=torch.randn(1, self.num_patches+1, embedding_dim),
                                               requires_grad=True)

        # 7. Create embedding dropout value
        self.embedding_dropout = nn.Dropout(p=embedding_dropout)

        # 8. Create patch embedding layer
        self.patch_embedding = PatchEmbedding(in_channels=in_channels,
                                              patch_size=patch_size,
                                              embedding_dim=embedding_dim)

        # 9. Create Transformer Encoder blocks (we can stack Transformer Encoder blocks using nn.Sequential())
        # Note: The "*" means "all"
        self.transformer_encoder = nn.Sequential(*[TransformerEncoderBlock(embedding_dim=embedding_dim,
                                                                            num_heads=num_heads,
                                                                            mlp_size=mlp_size,
                                                                            mlp_dropout=mlp_dropout) for _ in range(num_transformer_layers)])

        # 10. Create classifier head
        self.classifier = nn.Sequential(
            nn.LayerNorm(normalized_shape=embedding_dim),
            nn.Linear(in_features=embedding_dim,
                      out_features=num_classes)
        )

    # 11. Create a forward() method
    def forward(self, x):

        # 12. Get batch size
        batch_size =  x.shape[0]

        # 13. Create class token embedding and expand it to match the batch size (equation 1)
        class_token = self.class_embedding.expand(batch_size, -1, -1) # "-1" means to infer the dimension (try this line on its own)

        # 14. Create patch embedding (equation 1)
        x = self.patch_embedding(x)

        # 15. Concat class embedding and patch embedding (equation 1)
        x = torch.cat((class_token, x), dim=1)

        # 16. Add position embedding to patch embedding (equation 1)
        x = self.position_embedding + x

        # 17. Run embedding dropout (Appendix B.1)
        x = self.embedding_dropout(x)

        # 18. Pass patch, position and class embedding through transformer encoder layers (equations 2 & 3)
        x = self.transformer_encoder(x)

        # 19. Put 0 index logit through classifier (equation 4)
        x = self.classifier(x[:, 0]) # run on each sample in a batch at 0 index

        return x

In [24]:
# Train our MOdel

# Create an instance of ViT with the number of classes we're working with (pizza, steak, sushi)
#vit = ViT(num_classes=len(class_names))
vit = ViT()
vit.to(device)

ViT(
  (embedding_dropout): Dropout(p=0.1, inplace=False)
  (patch_embedding): PatchEmbedding(
    (patcher): Conv2d(1, 64, kernel_size=(8, 8), stride=(8, 8))
    (flatten): Flatten(start_dim=2, end_dim=3)
  )
  (transformer_encoder): Sequential(
    (0): TransformerEncoderBlock(
      (msa_block): MultiheadSelfAttentionBlock(
        (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
      )
      (mlp_block): MLPBlock(
        (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=512, out_features=128, bias=True)
          (4): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (1): 

In [28]:
# Training loop
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm, trange

optimizer = Adam(vit.parameters(), lr=LEARNING_RATE)
criterion = CrossEntropyLoss()
for epoch in trange(EPOCHS, desc="Training"):
    train_loss = 0.0
    vit.train()
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} in training", leave=False):
        #x, y = batch
        #x, y = x.to(device), y.to(device)
        x = batch['image'].to(device)
        y = batch['label'].to(device)
        y_hat = vit(x)
        loss = criterion(y_hat, y)

        train_loss += loss.detach().cpu().item() / len(train_loader)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{EPOCHS} loss: {train_loss:.4f}")
    # Validation Phase
    vit.eval()  # Set model to evaluation mode
    with torch.no_grad():
        correct, total = 0, 0
        test_loss = 0.0
        for batch in tqdm(val_loader, desc="Testing"):
            x = batch['image'].to(device)
            y = batch['label'].to(device)
            y_hat = vit(x)
            loss = criterion(y_hat, y)
            test_loss += loss.detach().cpu().item() / len(val_loader)

            correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
            total += len(x)
        print(f"Test loss: {test_loss:.2f}")
        print(f"Test accuracy: {correct / total * 100:.2f}%")

Epoch 1 in training: 100%|█████████▉| 680/681 [00:52<00:00, 13.17it/s]
                                                                      

Epoch 1/40 loss: 0.6646



Training:   2%|▎         | 1/40 [01:08<44:12, 68.01s/it]

Test loss: 0.66
Test accuracy: 61.45%



Epoch 2 in training: 100%|██████████| 681/681 [00:51<00:00, 13.21it/s]
                                                                      

...

Test loss: 0.57
Test accuracy: 71.22%



Epoch 40 in training: 100%|██████████| 681/681 [00:53<00:00, 12.55it/s]
                                                                       

Epoch 40/40 loss: 0.5666



Training: 100%|██████████| 40/40 [46:53<00:00, 70.34s/it]

Test loss: 0.57
Test accuracy: 71.46%


In [29]:
# Test loop
with torch.no_grad():
    correct, total = 0, 0
    test_loss = 0.0
    for batch in tqdm(test_loader, desc="Testing"):
        x = batch['image'].to(device)
        y = batch['label'].to(device)
        y_hat = vit(x)
        loss = criterion(y_hat, y)
        test_loss += loss.detach().cpu().item() / len(test_loader)

        correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
        total += len(x)
    print(f"Test loss: {test_loss:.2f}")
    print(f"Test accuracy: {correct / total * 100:.2f}%")

Testing: 100%|██████████| 195/195 [00:04<00:00, 46.82it/s]

Test loss: 0.57
Test accuracy: 70.99%
